In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options



import time
import pandas as pd

### 설정
- **review_targets변수 name, url변경해서 실행!**

In [38]:
# 수집할 리뷰 페이지 URL 목록
store_list = [
    {'name':'방앗간_담음', 'url': ["https://smartstore.naver.com/1976house/products/4808940703", "https://smartstore.naver.com/1976house/products/5227028199", "https://smartstore.naver.com/1976house/products/4808851813"]}, 
    {'name':'방앗간아이', 'url': ['https://smartstore.naver.com/khtfire/products/309312893']}, 
    {'name':'전라도청년', 'url': ['https://smartstore.naver.com/jeolla-youth/products/7851205144?NaPm=ct%3Dmdemcf1g%7Cci%3DER92c90ef4%2D6706%2D11f0%2D8f70%2D9625e2775ece%7Ctr%3Dplan%7Chk%3D82d5b1fd1c569a7eabdcd435a120f8aa31d7e826%7Cnacn%3DivSrBsQLKEmz']}
]
# 검색결과 iframe으로 전환
# driver.switch_to.frame("searchIframe")
# time.sleep(2)

### 실행

In [40]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def crawl_naver_store_reviews(store_name, store_url, max_clicks=350):
    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    driver.get(store_url)
    time.sleep(3)

    rows = []  # ← 한 리뷰당 하나의 딕셔너리(한 행)
    click_count = 0

    try:
        # ✅ 리뷰 탭 클릭
        review_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-name="REVIEW"]'))
        )
        review_tab.click()
        time.sleep(3)

        while click_count < max_clicks:
            # ✅ 리뷰 박스 찾기
            review_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "_2g7PKvqCKe"))
            )

            ul = review_box.find_element(By.TAG_NAME, "ul")
            li_list = ul.find_elements(By.TAG_NAME, "li")

            for li in li_list:
                try:
                    # ✅ 리뷰 텍스트 추출
                    review_text = ""
                    try:
                        review_text = li.find_element(By.CLASS_NAME, "_3z6gI4oI6l").text.strip()
                    except:
                        pass

                    # ✅ 평점 추출
                    rating = ""
                    try:
                        rating = li.find_element(By.CSS_SELECTOR, 'em._15NU42F3kT').text.strip()
                    except:
                        pass

                    # ✅ 작성 날짜 추출
                    date = ""
                    try:
                        date = li.find_element(By.CSS_SELECTOR, 'span._2L3vDiadT9').text.strip()
                    except:
                        pass

                    # ✅ 결과 저장
                    rows.append({
                        "가게이름": store_name,
                        "리뷰내용": review_text,
                        "평점": rating,
                        "작성일": date
                    })

                except Exception as e:
                    print("❗ 리뷰 수집 실패:", e)
                    continue

            print(f"[{store_name}] 클릭 {click_count + 1}회차 수집 완료 (누적 리뷰: {len(rows)})")

            # ✅ 다음 버튼 클릭
            try:
                next_btn = driver.find_element(By.CSS_SELECTOR, 'a[role="button"]._2Ar8-aEUTq')
                is_hidden = next_btn.get_attribute("aria-hidden")

                if is_hidden == "true":
                    print("📌 마지막 페이지입니다. 종료.")
                    break
                else:
                    next_btn.click()
                    click_count += 1
                    time.sleep(2)

            except Exception as e:
                print("📌 다음 버튼 클릭 실패 또는 존재하지 않음:", e)
                break

    except Exception as e:
        print(f"[{store_name}] 크롤링 중 오류:", e)

    driver.quit()

    return rows 

In [41]:
results = []

for store in store_list:
    store_name = store["name"]
    urls = store["url"]

    for idx, url in enumerate(urls):
        print(f"🚀 {store_name} - 상품 {idx + 1} 수집 중...")
        reviews = crawl_naver_store_reviews(f"{store_name}_{idx + 1}", url)  # 리뷰 리스트 반환
        results.extend(reviews)  # ⬅ 리스트를 병합

# 리뷰 하나당 하나의 행이 되도록
df = pd.DataFrame(results)
df.to_csv("naver_store_reviews.csv", index=False, encoding="utf-8-sig")
print("✅ 모든 리뷰 저장 완료!")

🚀 방앗간_담음 - 상품 1 수집 중...
[방앗간_담음_1] 클릭 1회차 수집 완료 (누적 리뷰: 20)
[방앗간_담음_1] 클릭 2회차 수집 완료 (누적 리뷰: 40)
[방앗간_담음_1] 클릭 3회차 수집 완료 (누적 리뷰: 60)
[방앗간_담음_1] 클릭 4회차 수집 완료 (누적 리뷰: 80)
[방앗간_담음_1] 클릭 5회차 수집 완료 (누적 리뷰: 100)
[방앗간_담음_1] 클릭 6회차 수집 완료 (누적 리뷰: 120)
[방앗간_담음_1] 클릭 7회차 수집 완료 (누적 리뷰: 140)
[방앗간_담음_1] 클릭 8회차 수집 완료 (누적 리뷰: 160)
[방앗간_담음_1] 클릭 9회차 수집 완료 (누적 리뷰: 165)
📌 마지막 페이지입니다. 종료.
🚀 방앗간_담음 - 상품 2 수집 중...
[방앗간_담음_2] 클릭 1회차 수집 완료 (누적 리뷰: 20)
[방앗간_담음_2] 클릭 2회차 수집 완료 (누적 리뷰: 25)
📌 마지막 페이지입니다. 종료.
🚀 방앗간_담음 - 상품 3 수집 중...
[방앗간_담음_3] 클릭 1회차 수집 완료 (누적 리뷰: 20)
[방앗간_담음_3] 클릭 2회차 수집 완료 (누적 리뷰: 40)
[방앗간_담음_3] 클릭 3회차 수집 완료 (누적 리뷰: 60)
[방앗간_담음_3] 클릭 4회차 수집 완료 (누적 리뷰: 65)
📌 마지막 페이지입니다. 종료.
🚀 방앗간아이 - 상품 1 수집 중...
[방앗간아이_1] 클릭 1회차 수집 완료 (누적 리뷰: 20)
[방앗간아이_1] 클릭 2회차 수집 완료 (누적 리뷰: 40)
[방앗간아이_1] 클릭 3회차 수집 완료 (누적 리뷰: 60)
[방앗간아이_1] 클릭 4회차 수집 완료 (누적 리뷰: 80)
[방앗간아이_1] 클릭 5회차 수집 완료 (누적 리뷰: 100)
[방앗간아이_1] 클릭 6회차 수집 완료 (누적 리뷰: 120)
[방앗간아이_1] 클릭 7회차 수집 완료 (누적 리뷰: 140)
[방앗간아이_1] 클릭 8회차 수집 완료 (누적 리뷰: 160)
[방앗간아이_1] 클릭 9회차 수집 완료

In [44]:
len(df)

10271

In [45]:
df

,가게이름,리뷰내용,평점,작성일
0,방앗간_담음_1,재구매참기름 고소하고 넘 맛납니다. 두번째구매해요.앞으로도 쭈욱~~,5,25.06.29.
1,방앗간_담음_1,포장도 꼼꼼하고 몇년째 믿고 먹는집이에요~~,5,25.07.12.
2,방앗간_담음_1,국산 참기름 믿을 수 있고 맛도 좋아요!!,5,25.07.10.
3,방앗간_담음_1,재구매다른 곳 꺼는 못 먹겠어요.\n향 좋고 맛있어요.,5,25.06.11.
4,방앗간_담음_1,재구매항상 믿고 주문하고있습니다,5,25.06.08.
...,...,...,...,...
10266,전라도청년_1,빨리와서 좋아욤♡ 깔끔포장!! 굳~,5,23.01.19.
10267,전라도청년_1,포장꼼꼼하네요~~~^^,4,23.01.20.
10268,전라도청년_1,맛있어요.......,3,23.01.24.
10269,전라도청년_1,좋아요~~~~~~~~,5,23.01.25.
